In [3]:
import numpy as np
from typing import List, Dict, Union, Tuple
from tenpy.networks.site import SpinSite
from tenpy.linalg import np_conserved as npc
from tenpy.linalg.np_conserved import Array
from tenpy.networks.mpo import MPO
from tenpy.models.model import MPOModel
from tenpy.models.lattice import Chain
from tenpy.networks.mps import MPS  
from tenpy.algorithms.network_contractor import ncon

In [1]:
# --------- Dirichlet utility (explicit, stable) ---------
#Need a vector in R^n that sums to 1 so that then we draw randomly a sum and the multiplication between this dirichlet vector and the sum is the substocastic row/column

def _dirichlet_weight(rng: np.random.Generator, n: int, alpha: float = 1.0) -> np.ndarray:
    """
    Return w ∈ R^n, w ≥ 0, sum(w) = 1.
    Samples w via independent Gamma(alpha,1) and normalizing.
    - alpha = 1.0  → uniform over the simplex (flat with no bias).
    - alpha < 1.0  → spikier (few large entries, many near zero).
    - alpha > 1.0  → smoother (entries more even).
    Edge cases:
      n = 0 → empty vector, n = 1 → [1.0].
    """

    #Sanity checks to avoid useless computation (R^0 and R vectors)
    if n == 0:
        return np.empty((0,), dtype=np.float64)
    if n == 1:
        return np.array([1.0], dtype=np.float64)
    #Invalid parameter
    if alpha <= 0:
        raise ValueError("dirichlet_alpha must be > 0.")
    
    
    #Gamma sampled vector of R^n
    g = rng.gamma(shape=alpha, scale=1.0, size=n)
    
    #Sum of the vector 
    s = g.sum()
    
    # numerically, s > 0 almost surely for alpha>0; still guard:
    if s == 0.0:
        # extremely unlikely; fallback to a one-hot at a random index
        w = np.zeros(n, dtype=np.float64)
        w[rng.integers(n)] = 1.0
        return w
    return g / s


# --------- Validators (keep constraints honest) ----------

def _check_L_row_substochastic(L: np.ndarray, atol: float = 1e-12):
    # For each (x,i): sum_j L[x,i,j] ≤ 1
    row_sums = L.sum(axis=2)  # (d, Dl)
    if np.any(row_sums > 1 + atol):
        raise AssertionError("random_L: found a row with sum > 1.")
    if np.any(L < -atol) or np.any(L > 1 + atol):
        raise AssertionError("random_L: entries must lie in [0,1].")

def _check_R_col_substochastic(R: np.ndarray, atol: float = 1e-12):
    # For each (x,j): sum_i R[x,i,j] ≤ 1
    col_sums = R.sum(axis=1)  # (d, Dr)
    if np.any(col_sums > 1 + atol):
        raise AssertionError("random_R: found a column with sum > 1.")
    if np.any(R < -atol) or np.any(R > 1 + atol):
        raise AssertionError("random_R: entries must lie in [0,1].")

def _check_box_0_1(A: np.ndarray, atol: float = 1e-12):
    if np.any(A < -atol) or np.any(A > 1 + atol):
        raise AssertionError("random_A1: entries must lie in [0,1].")

# --------- Substochastic generators (switched constraints) ---------
def random_L_sub(
    d: int, Dl: int, Dr: int, seed=None, *,
    dirichlet_alpha: float = 1.0, check: bool = True
) -> np.ndarray:
    """
    Substochastic L_k, shape (d, Dl, Dr).
    Constraint : for each physical index x and each ROW i,
        sum_j L[x, i, j] ≤ 1  (row-substochastic)
    Construction: for each (x,i), draw a total mass s ~ U[0,1) and a Dirichlet
    weight w over Dr entries; set the row L[x,i,:] = s * w.
    """
    rng = np.random.default_rng(seed)
    L = np.zeros((d, Dl, Dr), dtype=np.float64)
    
    #Avoid unnecessary computation if degenerate input
    if d == 0 or Dl == 0 or Dr == 0:
        return L

    #Populate the rows substocastically
    for x in range(d):
        for i in range(Dl):
            s = rng.random()  # total mass in [0,1)
            if s == 0.0:
                continue      # zero row
            w = _dirichlet_weight(rng, Dr, alpha=dirichlet_alpha)  # sums to 1
            L[x, i, :] = s * w                                    # row sum = s ≤ 1

    if check:
        _check_L_row_substochastic(L)
    return L

def random_R_sub(
    d: int, Dl: int, Dr: int, seed=None, *,
    dirichlet_alpha: float = 1.0, check: bool = True
) -> np.ndarray:
    """
    Substochastic R_k, shape (d, Dl, Dr).
    Constraint : for each physical x and each COLUMN j,
        sum_i R[x, i, j] ≤ 1  (column-substochastic)
    Construction: for each (x,j), draw mass s ~ U[0,1) and a Dirichlet over Dl
    entries; set the column R[x, :, j] = s * w.
    """
    rng = np.random.default_rng(seed)
    R = np.zeros((d, Dl, Dr), dtype=np.float64)
    
    #Avoid unnecessary computation if degenerate input
    if d == 0 or Dl == 0 or Dr == 0:
        return R
    
    #Populate the columns substocastically
    for x in range(d):
        for j in range(Dr):
            s = rng.random()  # total mass in [0,1)
            if s == 0.0:
                continue      # zero column
            w = _dirichlet_weight(rng, Dl, alpha=dirichlet_alpha)  # sums to 1
            R[x, :, j] = s * w                                    # column sum = s ≤ 1

    if check:
        _check_R_col_substochastic(R)
    return R

def random_A_sub(d: int, D: int, seed=None, *, check: bool = True) -> np.ndarray:
    """
    First site core A1, shape (d, 1, D).
    Continuous in [0,1] (uniform). To have occasional exact 1.0 values,
    we can optionally snap a tiny fraction to 1.0 after sampling.
    """
    rng = np.random.default_rng(seed)
    A = rng.uniform(0.0, 1.0, size=(d, 1, D)).astype(np.float64)  # in [0,1)
    
    # Optional: snap some values to 1.0 with tiny prob (disabled by default)
    # snap_mask = rng.random(A.shape) < 1e-4
    # A[snap_mask] = 1.0
    
    if check:
        _check_box_0_1(A)
    return A



NameError: name 'np' is not defined